In [15]:
import Pkg

Pkg.add("HiddenMarkovModels")
Pkg.add("Distributions")
Pkg.add("StableRNGs")


   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
   Installed StableRNGs ─ v1.0.1
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [860ef19b] + StableRNGs v1.0.1
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [860ef19b] + StableRNGs v1.0.1
Precompiling project...
  ✓ StableRNGs
  1 dependency successfully precompiled in 3 seconds. 394 already precompiled.


In [16]:
using HiddenMarkovModels, Distributions
using LinearAlgebra
using StableRNGs

In [17]:
rng = StableRNG(63);

In [18]:
init = [0.6, 0.4]
trans = [0.7 0.3; 0.2 0.8]
dists = [MvNormal([-0.5, -0.8], I), MvNormal([0.5, 0.8], I)]
hmm = HMM(init, trans, dists)

Hidden Markov Model with:
 - initialization: [0.6, 0.4]
 - transition matrix: [0.7 0.3; 0.2 0.8]
 - observation distributions: [IsoNormal(
dim: 2
μ: [-0.5, -0.8]
Σ: [1.0 0.0; 0.0 1.0]
)
, IsoNormal(
dim: 2
μ: [0.5, 0.8]
Σ: [1.0 0.0; 0.0 1.0]
)
]

In [19]:

init_guess = [0.5, 0.5]
trans_guess = [0.6 0.4; 0.3 0.7]
dists_guess = [MvNormal([-0.4, -0.7], I), MvNormal([0.4, 0.7], I)]
hmm_guess = HMM(init_guess, trans_guess, dists_guess);

In [22]:
cos, long_obs_seq = rand(rng, hmm, 200)

(state_seq = [1, 1, 1, 2, 1, 1, 1, 1, 1, 2  …  2, 2, 2, 2, 2, 2, 2, 1, 1, 1], obs_seq = [[-0.9519717318231875, -1.0102087593571034], [0.25930390607903675, -1.3459438609375318], [-2.4471322095519072, -1.0389249448197877], [0.1128777317927428, -0.3763664473545407], [0.0054566966495605, 0.33349441502245125], [-0.6055371255072928, -0.9377027235074087], [-0.8208596268628077, -1.45902827964301], [0.14417125211797444, -0.19127045147920596], [-0.1314378318233967, -1.247202792876466], [0.10848359617775932, 1.6927319492959723]  …  [0.6796445288658193, 0.8460945730121384], [-0.37468502793829617, -0.07541043213907905], [-1.0269083767159222, 0.6056173631866001], [0.36415310562507086, 1.1069233174659536], [-0.4196495941383094, -0.5334785935933555], [0.05444944174881672, 1.2565915704688677], [1.0080731730045807, -0.8958295615218537], [-0.16498929335700985, -1.030308491112538], [-1.196120499237034, -1.714832575035934], [-0.899630444716361, -1.0220415869196988]])

In [35]:
size(long_obs_seq)

(200,)

In [32]:
first(long_obs_seq)

2-element Vector{Float64}:
 -0.9519717318231875
 -1.0102087593571034

In [25]:
hmm_est, loglikelihood_evolution = baum_welch(hmm_guess, long_obs_seq);

In [37]:
cat(transition_matrix(hmm_est), transition_matrix(hmm); dims=3)

2×2×2 Array{Float64, 3}:
[:, :, 1] =
 0.774461  0.225539
 0.244659  0.755341

[:, :, 2] =
 0.7  0.3
 0.2  0.8

In [36]:
map(mean, hcat(obs_distributions(hmm_est), obs_distributions(hmm)))

2×2 Matrix{Vector{Float64}}:
 [-0.508898, -0.844124]  [-0.5, -0.8]
 [0.444826, 0.969582]    [0.5, 0.8]

In [38]:
nb_seqs = 100
long_obs_seqs = [last(rand(rng, hmm, rand(rng, 100:200))) for k in 1:nb_seqs];
typeof(long_obs_seqs)

Vector{Vector{Vector{Float64}}} (alias for Array{Array{Array{Float64, 1}, 1}, 1})

In [46]:
size(long_obs_seqs[1])

(141,)

In [49]:
long_obs_seqs[3]

140-element Vector{Vector{Float64}}:
 [-3.1567126734317608, 0.34171428354817124]
 [0.9865398185448774, 0.4790982970111405]
 [-0.25494993969858815, 1.6512797456159363]
 [-0.01305711100509599, 1.9948552780275086]
 [0.23767512252680428, 0.3020680690956243]
 [0.0926661169606317, -0.2967408321465761]
 [0.7661011590061635, 1.209547680315247]
 [0.3713196395503653, 0.14813897918434193]
 [1.1361417800299305, 3.1006181679428586]
 [0.9408228600951382, 3.072771515411892]
 ⋮
 [0.614687730842862, 1.2701527606186904]
 [-0.6479937675597304, -0.9339845188314926]
 [-2.168517383517041, -2.3163333234826218]
 [0.006701120588462506, -0.8217032389757033]
 [-1.6893900788246798, -1.3906615926385149]
 [-2.4728240132201127, -0.6043483132491939]
 [0.12253067609037482, 0.09662730677425879]
 [-1.7423046164778768, -1.8440731935709378]
 [-1.5596586063095825, -0.15772841351864564]